- Queryの最後の単語
- 一番大きなboxに対して、その半分以下のboxは無視する -> すると、どのくらいのボックスがあるか？

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install swifter
!pip install dgl-cu100
!pip install pytorch-metric-learning

In [0]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import time
import gc
from glob import glob
from tqdm.notebook import tqdm
import pickle
import logging
import json
import math

import base64
import swifter
import gensim

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

pd.set_option('display.max_columns', 500)

In [4]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

import keras
from keras.preprocessing.sequence import pad_sequences

import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GraphConv

from pytorch_metric_learning import losses

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

print(transformers.__version__)

Using TensorFlow backend.
Using backend: pytorch
ERROR:root:The testing module requires faiss. You can install the GPU version with the command 'conda install faiss-gpu -c pytorch' 
                        or the CPU version with 'conda install faiss-cpu -c pytorch'. Learn more at https://github.com/facebookresearch/faiss/blob/master/INSTALL.md


2.9.0


In [5]:
os.getcwd()

'/content'

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/data/'
LOG_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/log/'
W2V_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/w2v/'
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/model/'
SUBMIT_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/prediction_result/'

In [0]:
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/data/'
# LOG_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/log/'
# W2V_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/w2v/'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT/'
# MODEL_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/model/'

### Read Data

In [8]:
train = pd.read_csv(DATA_DIR+'train/train.sample.tsv', sep='\t')
print(train.shape)
train.head()

(10000, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,AAAAAAAAQEAAAKlDAABPQwAAb0MAALBBAICmQwAATEM=,AAAAAAAAAAAnNsFAAAAAAOeVGD0AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAAA==,treble popular reed,923271
1,101783080,80,61,1,AACgQAAAoEAAAJZCAABgQg==,AAAAADnKfjyatb8/AAAAALxPBz0MH6E9xwkoPAAAAAAAAA...,HAAAAAAAAAA=,compatible ink cartridges,234031
2,100492797,750,750,1,AAAsQgCAg0MAgCREAADqQw==,AAAAAAAAAABmUlE/AAAAABvc0jwAAAAAAAAAAAAAAAAAAA...,BwAAAAAAAAA=,check new look facial cream,199926
3,101708088,480,640,5,AAB6QwCA0kMAANBDAMAeRAAAfEMAAKBAAIDWQwAAVkMAAH...,AAAAAAAAAAAAAAAAAAAAAAXyfz4AAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAcAAAAAAAAABwAAAAAAAAAHAAAAAAAAAAcAA...,royal commemorative stamp,772725
4,100316842,800,800,4,AAAKQwCAgEMAADNEAAAHRAAAIkMAwCZEAACqQwDARkQAAE...,AAAAADSdaTwVTdA8AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAGAAAAAAAAAAYAAAAAAAAAHAAAAAAAAAA=,calendula baby shower gel,362576


In [9]:
valid = pd.read_csv(DATA_DIR+'valid/valid.tsv', sep='\t')
print(valid.shape)
valid.head()

(14720, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103047314,800,800,2,AACoQgAA5EIAAKdDAIDuQwCAq0MAAJBCAMA3RABAHkQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
1,103030678,800,800,9,AIDRQwAAoEAAwEJEAABpQwAAA0MAgMlDAABHRAAAIkQAgL...,AAAAAAAAAACdFBI7AAAAAAAAAABu+j0+AAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAAHQAAAAAAAAAEAA...,breathable and comfortable children's shoes,0
2,103031217,488,750,2,AABGQwAANEIAgNNDAADsQwAAUEIAALVDAADRQwCALEQ=,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0
3,103026621,929,800,10,AACtQwCAA0QAgPxDAIAoRAAAekMAALdDAACwQwCA30MAgK...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,HAAAAAAAAAAdAAAAAAAAAAAAAAAAAAAACQAAAAAAAAAJAA...,breathable and comfortable children's shoes,0
4,103017048,800,800,2,AAA+QwCAi0MAgAtEAMA0RAAAqEMAAHhCAMA1RABADUQ=,AAAAAAAAAADIb2s9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAAEAAAAAAAAAA==,breathable and comfortable children's shoes,0


In [10]:
testA = pd.read_csv(DATA_DIR+'testA/testA.tsv', sep='\t')
print(testA.shape)
testA.head()

(28830, 9)


,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,103052445,798,798,1,AIC3QwAAF0MAABxEAMAtRA==,AAAAAAAAAABwKQE9AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
1,103044513,800,800,1,AABqQwAAzEIAADJEAMAgRA==,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0
2,103035030,800,800,8,AAAAQwAAAEEAgNRDAICGQwAAUEEAAPBBAACUQgAA3EIAAI...,AAAAAAAAAABFkBg+AAAAAM5daTwAAAAAAAAAAAAAAAAAAA...,AAAAAAAAAAAFAAAAAAAAAAIAAAAAAAAAAwAAAAAAAAAEAA...,straw heavy-bottomed sandals,0
3,103036197,800,800,3,AAAmQwAAwEEAwC1EAICqQwAAJ0MAAFJDAAA1RADAAkQAAE...,AAAAAAAAAAD2rhE+AAAAAAAAAADbLAw+AAAAANGzLD4AAA...,BAAAAAAAAAAEAAAAAAAAAAQAAAAAAAAA,straw heavy-bottomed sandals,0
4,103055323,800,800,1,AABUQwAA+EEAgChEAEBCRA==,AAAAAKzRGTs0osc+kJpVPAAAAAAAAAAAAAAAAAAAAAAAAA...,BAAAAAAAAAA=,straw heavy-bottomed sandals,0


### Decode Data

In [0]:
def transform_boxes(df_row):
    return np.frombuffer(base64.b64decode(df_row.boxes), dtype=np.float32).reshape(df_row.num_boxes, 4)

def transform_features(df_row):
    return np.frombuffer(base64.b64decode(df_row.features), dtype=np.float32).reshape(df_row.num_boxes, 2048)

def transform_class_labels(df_row):
    return np.frombuffer(base64.b64decode(df_row.class_labels), dtype=np.int64).reshape(df_row.num_boxes)

def transform_df(df):
    df.boxes = df.swifter.apply(transform_boxes, axis=1)
    df.features = df.swifter.apply(transform_features, axis=1)
    df.class_labels = df.swifter.apply(transform_class_labels, axis=1)
    return df

In [12]:
train = transform_df(train)
valid = transform_df(valid)
testA = transform_df(testA)

In [13]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576


### Feature Engineering

- box area

In [14]:
train['box_normalized'] = train.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
valid['box_normalized'] = valid.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
testA['box_normalized'] = testA.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)

In [15]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[28, 28]",treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220..."
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[28],compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803..."
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...",[7],check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8..."
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[28, 28, 28, 28, 28]",royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666..."
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[28, 6, 6, 28]",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8..."


In [0]:
# box area
train['box_area_normalized'] = train.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
valid['box_area_normalized'] = valid.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
testA['box_area_normalized'] = testA.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))

In [0]:
train['box_area_normalized_2'] = train.box_area_normalized.apply(lambda x: np.array([xi/x.max() if xi/x.max()>=0.2 else 0 for xi in x]))
valid['box_area_normalized_2'] = valid.box_area_normalized.apply(lambda x: np.array([xi/x.max() if xi/x.max()>=0.2 else 0 for xi in x]))
testA['box_area_normalized_2'] = testA.box_area_normalized.apply(lambda x: np.array([xi/x.max() if xi/x.max()>=0.2 else 0 for xi in x]))

- word2vec of box labels

In [18]:
class_label_dict = {
    0:'top clothes (coat, jacket, shirt, etc.)',
    1:'skirt & dress',
    2:'bottom clothes (trousers, pants, etc.)',
    3:'luggage, leather goods',
    4:'shoes',
    5:'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
    6:'snacks, nuts, liquor and tea',
    7:'makeup, perfume, beauty tools and essential oils',
    8:'bottle drink',
    9:'furniture',
    10:'stationery',
    11:'household electrical appliances',
    12:'home decoration',
    13:'household fabric',
    14:'kitchenware',
    15:'home / personal cleaning tools',
    16:'storage supplies',
    17:'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
    18:'outdoor product',
    19:'lighting',
    20:'toys',
    21:'underwear',
    22:'digital supplies',
    23:'bed linens',
    24:'baby products',
    25:'personal care',
    26:'sporting goods',
    27:'clothes (accessories, baby clothing, etc.)',
    28:'others',
    29:'human face',
    30:'arm',
    31:'hair',
    32:'hand',
}

train['class_labels_words'] = train.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
valid['class_labels_words'] = valid.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
testA['class_labels_words'] = testA.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))

train['class_labels'] = train.class_labels.swifter.apply(lambda x: np.array([class_label_dict[xi] for xi in x]))
valid['class_labels'] = valid.class_labels.swifter.apply(lambda x: np.array([class_label_dict[xi] for xi in x]))
testA['class_labels'] = testA.class_labels.swifter.apply(lambda x: np.array([class_label_dict[xi] for xi in x]))

In [19]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized,box_area_normalized,box_area_normalized_2,class_labels_words
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[others, others]",treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220...","[0.9760765550239234, 0.2421788737578212]","[1.0, 0.24811463046757173]",others others
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[others],compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803...",[0.7315573770491803],[1.0],others
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","[makeup, perfume, beauty tools and essential o...",check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8...",[0.2241333333333333],[1.0],"makeup, perfume, beauty tools and essential oils"
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[others, others, others, others, others]",royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666...","[0.11563802083333333, 0.120419921875, 0.133255...","[0.8677936290795388, 0.9036789134258356, 1.0, ...",others others others others others
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[others, snacks, nuts, liquor and tea, snacks,...",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8...","[0.25558437500000003, 0.0356, 0.03279375, 0.21...","[1.0, 0.0, 0.0, 0.8450609510068836]","others snacks, nuts, liquor and tea snacks, nu..."


In [0]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path, encoding='utf-8') as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():  # word_indexのwordに対応するembがあれば、embを代入する
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words


def load_word_embed(word_embed_glove=W2V_DIR+"glove.840B.300d.txt", 
                    word_embed_crawl=W2V_DIR+"crawl-300d-2M.vec",
                    save_filename=W2V_DIR+"word_embedding_matrix_testA",
                    word_index=None):
    
    # Tokneizerの学習
    Tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
    Tokenizer.fit_on_texts(list(train['query'])+list(valid['query'])+list(testA['query'])
                           +list(train['class_labels_words'])+list(valid['class_labels_words'])+list(testA['class_labels_words']))
    
    if os.path.exists(save_filename + ".npy"):
        embedding_matrix = np.load(save_filename + ".npy").astype("float32")
    else:
        
        if word_index is None:
            word_index = Tokenizer.word_index
        
        glove_matrix, unknown_words_glove = build_matrix(word_index, word_embed_glove)
        print('n unknown words (glove): ', len(unknown_words_glove))
        
        crawl_matrix, unknown_words_crawl = build_matrix(word_index, word_embed_crawl)
        print('n unknown words (crawl): ', len(unknown_words_crawl))
        
        embedding_matrix = crawl_matrix + glove_matrix  
        np.save(save_filename, embedding_matrix)

        del crawl_matrix
        del glove_matrix
        gc.collect()
        
    return embedding_matrix, Tokenizer


def calc_mean_w2v(class_labels):
    class_labels_index = tokenizer_w2v.texts_to_sequences([class_labels])
    return np.mean(embedding_matrix[class_labels_index], axis=0)
    
    # try:
    #     return embedding_matrix[class_labels_index]
    #     #return np.mean(embedding_matrix[class_labels_index], axis=0)
    # except KeyError:
    #     print('Error')
    #     return np.zeros(300)

In [0]:
# about 5 minites
embedding_matrix, tokenizer_w2v = load_word_embed()

## Model

- initial setting

In [0]:
# make logger
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
sc = logging.StreamHandler()
logger.addHandler(sc)
fh = logging.FileHandler(LOG_DIR+'20200505.log')
logger.addHandler(fh)

In [0]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


### DataLoader

In [25]:
train.head()

,product_id,image_h,image_w,num_boxes,boxes,features,class_labels,query,query_id,box_normalized,box_area_normalized,box_area_normalized_2,class_labels_words
0,102851856,338,209,2,"[[0.0, 3.0, 338.0, 207.0], [239.0, 22.0, 333.0...","[[0.0, 0.0, 6.0378604, 0.0, 0.037252333, 0.0, ...","[others, others]",treble popular reed,923271,"[[0.0, 0.014354066985645933, 1.0, 0.9904306220...","[0.9760765550239234, 0.2421788737578212]","[1.0, 0.24811463046757173]",others others
1,101783080,80,61,1,"[[5.0, 5.0, 75.0, 56.0]]","[[0.0, 0.015551143, 1.4977295, 0.0, 0.03303502...",[others],compatible ink cartridges,234031,"[[0.0625, 0.08196721311475409, 0.9375, 0.91803...",[0.7315573770491803],[1.0],others
2,100492797,750,750,1,"[[43.0, 263.0, 658.0, 468.0]]","[[0.0, 0.0, 0.81766355, 0.0, 0.02573972, 0.0, ...","[makeup, perfume, beauty tools and essential o...",check new look facial cream,199926,"[[0.05733333333333333, 0.3506666666666667, 0.8...",[0.2241333333333333],[1.0],"makeup, perfume, beauty tools and essential oils"
3,101708088,480,640,5,"[[250.0, 421.0, 416.0, 635.0], [252.0, 5.0, 42...","[[0.0, 0.0, 0.0, 0.0, 0.24994667, 0.0, 0.0, 0....","[others, others, others, others, others]",royal commemorative stamp,772725,"[[0.5208333333333334, 0.6578125, 0.86666666666...","[0.11563802083333333, 0.120419921875, 0.133255...","[0.8677936290795388, 0.9036789134258356, 1.0, ...",others others others others others
4,100316842,800,800,4,"[[138.0, 257.0, 716.0, 540.0], [162.0, 667.0, ...","[[0.0, 0.014258672, 0.02542738, 0.0, 0.0, 0.0,...","[others, snacks, nuts, liquor and tea, snacks,...",calendula baby shower gel,362576,"[[0.1725, 0.32125, 0.895, 0.675], [0.2025, 0.8...","[0.25558437500000003, 0.0356, 0.03279375, 0.21...","[1.0, 0.0, 0.0, 0.8450609510068836]","others snacks, nuts, liquor and tea snacks, nu..."


In [26]:
l=[]
l+=[[1]]
l+=[[2]]
l

[[1], [2]]

In [0]:
MAX_LEN = 20
MAX_LEN_IMG = 5
MAX_LABEL_IMG = 8

class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.w2v_tokenizer = tokenizer_w2v
        #self.transform = transform
        self.train_mode = train_mode
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        ######################
        # query side
        ######################
        query = row['query']
        inputs_query = self.bert_tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        
        ids_query = inputs_query["input_ids"]
        token_type_ids_query = inputs_query["token_type_ids"]
        mask_query = inputs_query["attention_mask"]
        
        padding_len = MAX_LEN - len(ids_query)
        ids_query = ids_query + ([0]*padding_len)
        token_type_ids_query = token_type_ids_query + ([0]*padding_len)
        mask_query = mask_query + ([0]*padding_len)

        query = tokenizer_w2v.texts_to_sequences([query])
        query = pad_sequences(query, maxlen=MAX_LEN, padding='post')

        
        ######################
        # image side
        ######################

        # box_areaの大きい順にソート
        box_area = row['box_area_normalized']
        box_area_ordered = list(np.argsort(box_area)[::-1])
        #box_area_pos = list(np.where(box_area>0)[0])
        #box_area_ordered = [i for i in list(np.argsort(box_area)[::-1]) if i in box_area_pos]
        


        features_ordered = row.features[:MAX_LEN_IMG]
        box_ordered = row.box_normalized[:MAX_LEN_IMG]
        area_ordered = row.box_area_normalized[:MAX_LEN_IMG]
        class_labels_ordered = row.class_labels[:MAX_LEN_IMG]
        # features_ordered = row.features[box_area_ordered, :][:MAX_LEN_IMG]
        # box_ordered = row.box_normalized[box_area_ordered, :][:MAX_LEN_IMG]
        # area_ordered = row.box_area_normalized[box_area_ordered][:MAX_LEN_IMG]
        # class_labels_ordered = row.class_labels[box_area_ordered][:MAX_LEN_IMG]
        class_labels = []
        for cl in class_labels_ordered:
            class_labels += tokenizer_w2v.texts_to_sequences([cl])[0]


        features = pad_sequences([features_ordered.reshape(-1,2048)], maxlen=MAX_LEN_IMG, padding='post')[0]
        box = pad_sequences([box_ordered.reshape(-1,4)], maxlen=MAX_LEN_IMG, padding='post')[0]
        area = pad_sequences([area_ordered.reshape(-1,1)], maxlen=MAX_LEN_IMG, padding='post')[0]
        class_labels = pad_sequences([class_labels], maxlen=MAX_LABEL_IMG, padding='post')[0]

        
        if self.train_mode:
            return {
                'query': torch.tensor(query, dtype=torch.long),
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'features': torch.tensor(features, dtype=torch.float),
                'box': torch.tensor(box, dtype=torch.float),
                'area': torch.tensor(area, dtype=torch.float),
                'class_labels': torch.tensor(class_labels, dtype=torch.long).view(MAX_LABEL_IMG,1),
                # 'features_max': torch.tensor(features_max, dtype=torch.float),
                # 'box_max': torch.tensor(box_max, dtype=torch.float),
                # 'area_max': torch.tensor(area_max, dtype=torch.float),
                # 'class_labels_max': torch.tensor(class_labels, dtype=torch.long).view(1,1),
            }
        else:
            return {
                'query': torch.tensor(query, dtype=torch.long),
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'features': torch.tensor(features, dtype=torch.float),
                'box': torch.tensor(box, dtype=torch.float),
                'area': torch.tensor(area, dtype=torch.float),
                'class_labels': torch.tensor(class_labels, dtype=torch.long).view(MAX_LABEL_IMG,1),
                # 'features_max': torch.tensor(features_max, dtype=torch.float),
                # 'box_max': torch.tensor(box_max, dtype=torch.float),
                # 'area_max': torch.tensor(area_max, dtype=torch.float),
                # 'class_labels_max': torch.tensor(class_labels, dtype=torch.long).view(1,1),
            }

In [0]:
# # dataloaderの挙動確認

# dataset_train = KDDDataset(train, train_mode=True)
# train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True)

# for i, batch in enumerate(train_loader):
#     if i<1:
#         print(batch['query'].shape)
#         print(batch['features'].shape)
#         print(batch['box'].shape)
#         print(batch['area'].shape)
#         print(batch['class_labels'].shape)

#     else:
#         break

## Model

In [0]:
class PositionalEncoding(nn.Module):
 
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
 
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
 
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerModel(nn.Module):
 
    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        #self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        #self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        #self.decoder = nn.Linear(ninp, ntoken)
  
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
 
 
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
 
        #src = self.encoder(src) * math.sqrt(self.ninp)
        #src = self.pos_encoder(src)
        src = src * math.sqrt(self.ninp)
        output = self.transformer_encoder(src, self.src_mask)
        #output = self.decoder(output)
        return output

In [0]:
class QueryModel(nn.Module):
    def __init__(self):
        super(QueryModel, self).__init__()
        self.model_name = 'QueryModel'

        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        # for param in self.bert_model.parameters():
        #     param.requires_grad = True

        self.max_feature = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
      
        self.embedding = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.required_grad = True  # こっちもtrainableに

        self.lstm = nn.LSTM(self.embedding_size, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True) 


    def forward(self, query, ids, token_type_ids, mask):  
        
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids, attention_mask=mask) 
       
        out_query = self.embedding(query).squeeze()
        out_query, _  = self.lstm(out_query)
        out_query = out_query[:,0,:]
        
        out_query = torch.cat([pool_out, out_query], dim=-1)  # (batch, 768+128)
        
        return out_query

In [0]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.model_name = 'ImageModel'

        self.max_feature = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
        self.embedding = nn.Embedding(self.max_feature, self.embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.required_grad = True  # こっちもtrainableに

        self.conv1d = nn.Conv1d(in_channels=300, out_channels=128, kernel_size=4)

        self.fc1 = nn.Linear(2048+128, 512)
        self.fc2 = nn.Linear(5, 512)

        self.lstm = nn.LSTM(512+512, hidden_size=1024, num_layers=1, bidirectional=True, batch_first=True) 


    def forward(self, features, box, area, class_labels):

        class_labels = self.embedding(class_labels).squeeze()
        class_labels = self.conv1d(class_labels.permute(0,2,1)).permute(0,2,1)
        #class_labels = F.avg_pool1d(class_labels.transpose(1,2), kernel_size=class_labels.size()[1])

        # print('class_labels',class_labels.shape)
        # print('features',features.shape)
        out = torch.cat([class_labels, features], dim=-1)
        out = F.relu(self.fc1(out))

        out_pos = torch.cat([box, area], dim=-1)
        out_pos = F.relu(self.fc2(out_pos))
        out = torch.cat([out, out_pos], dim=-1)

        out, _ = self.lstm(out)
        out = F.avg_pool1d(out.transpose(1,2), kernel_size=out.size()[1]).squeeze()

        return out

In [0]:
class KDDModel(nn.Module):
    def __init__(self):
        super(KDDModel, self).__init__()
        self.model_name = 'KDDModel'

        self.model_query = QueryModel()
        self.model_image = ImageModel()

        self.fc1 = nn.Linear(3072, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 2)


    def forward(self, query, ids, token_type_ids, mask, features, box, area, class_labels):
        
        out_query = self.model_query(query, ids, token_type_ids, mask)
        out_image = self.model_image(features, box, area, class_labels)

        out = torch.cat([out_query, out_image], dim=-1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        
        return out.squeeze()

### Training

学習させるうえで
- クエリ側と画像側がある
- 正例と負例で、同じものを学習に入れてしまうと、同じものなのに、なぜラベルが異なるのかという感じになってしまう
- なので、ここでは、正例と負例は全く異なるサンプルで行うのが良い

In [0]:
def train_model(train_loader, model, optimizer, criterion): #, scheduler):
    model.train()
    avg_loss = 0.
    avg_score = 0.
    for idx, batch in enumerate(tqdm(train_loader)):
        batch_size = batch['query'].shape[0]
        #idx_neg = random.choices(range(batch_size), k=batch_size)
        
        idx_pos = random.sample(range(batch_size), k=batch_size//2)
        rest_set = list(set(range(batch_size)) - set(idx_pos))
        idx_neg1 = random.choices(rest_set, k=batch_size//2)
        idx_neg2 = random.choices(rest_set, k=batch_size//2)
        
        # idx_neg2 = idx_neg1[:]
        # random.shuffle(idx_neg2)
        #idx_neg1 = random.sample(range(batch_size), k=batch_size//2)
        #idx_neg2 = random.sample(range(batch_size), k=batch_size//2)
        #random.shuffle(idx_neg)

        query = torch.cat([batch['query'][idx_pos, :], batch['query'][idx_neg1, :]], dim=0).to(device)
        ids_query = torch.cat([batch['ids_query'][idx_pos, :], batch['ids_query'][idx_neg1, :]], dim=0).to(device)
        token_type_ids_query = torch.cat([batch['token_type_ids_query'][idx_pos, :], batch['token_type_ids_query'][idx_neg1, :]], dim=0).to(device)
        mask_query = torch.cat([batch['mask_query'][idx_pos, :], batch['mask_query'][idx_neg1, :]], dim=0).to(device)
        
        features = torch.cat([batch['features'][idx_pos, :], batch['features'][idx_neg2, :]]).to(device)
        box = torch.cat([batch['box'][idx_pos, :], batch['box'][idx_neg2, :]], dim=0).to(device)
        area = torch.cat([batch['area'][idx_pos, :], batch['area'][idx_neg2, :]], dim=0).to(device)
        class_labels = torch.cat([batch['class_labels'][idx_pos, :], batch['class_labels'][idx_neg2, :]], dim=0).to(device)

        
        labels = torch.cat([torch.tensor(np.ones(batch_size//2), dtype=torch.long), torch.tensor(np.zeros(batch_size//2), dtype=torch.long)], dim=0).to(device)


        out = model(query, ids_query, token_type_ids_query, mask_query, features, box, area, class_labels)
        loss = criterion(out, labels)

        _, predicted = torch.max(out, 1)
        correct = (predicted == labels).sum().item()
        acc = float(correct) / labels.size(0)
        print('acc', acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item() / len(train_loader)
        avg_score += acc / len(train_loader)

        del out,loss

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss, avg_score



def test_model(test_loader, model):
    model.eval()
    
    out_list = []   
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(test_loader)):
            query = batch['query'].to(device)
            ids_query = batch['ids_query'].to(device)
            token_type_ids_query = batch['token_type_ids_query'].to(device)
            mask_query = batch['mask_query'].to(device)
            features = batch['features'].to(device)
            box = batch['box'].to(device)
            area = batch['area'].to(device)
            class_labels = batch['class_labels'].to(device)

            out = F.softmax(model(query, ids_query, token_type_ids_query, mask_query, features, box, area, class_labels))[:,1]
            # out_query = model.model_query(ids_query, token_type_ids_query, mask_query)
            # out_image = model.model_image(graph)
            # out = F.cosine_similarity(out_query, out_image)

            out_list.append(out)
            del out
        
    torch.cuda.empty_cache()
    gc.collect()
    return out_list

In [0]:
# compute dcg@k for a single sample
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(3, r.size + 2)))
    return 0.


# compute ndcg@k (dcg@k / idcg@k) for a single sample
def get_ndcg(r, ref, k):
    dcg_max = dcg_at_k(ref, k)
    if not dcg_max:
        return 0.
    dcg = dcg_at_k(r, k)
    return dcg / dcg_max


def make_score(df_test, out_list):
    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    predictions = {}
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        predictions[f'{q_id}'] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values.astype(str)
    
    
    # read ground-truth
    reference = json.load(open(DATA_DIR+'valid/valid_answer.json'))

    # compute score for each query
    k = 5
    score_sum = 0.
    for qid in reference.keys():
        ground_truth_ids = set([str(pid) for pid in reference[qid]])
        ref_vec = [1.0] * len(ground_truth_ids)
        pred_vec = [1.0 if pid in ground_truth_ids else 0.0 for pid in predictions[qid]]
        score_sum += get_ndcg(pred_vec, ref_vec, k)
    # the higher score, the better
    score = score_sum / len(reference)
    
    return score

In [35]:
model = KDDModel().to(device)

# if device == 'cuda':
#     model = torch.nn.DataParallel(model) # make parallel
#     cudnn.benchmark = True


optimizer = torch.optim.Adam(model.parameters())

# optimizer = torch.optim.Adam([
#                         {'params': model.model_query.parameters()},
#                         {'params': model.model_image.parameters()}
#                     ])


criterion = nn.CrossEntropyLoss()
#criterion = nn.CosineSimilarity()
# criterion = nn.MSELoss()
#criterion = nn.CosineEmbeddingLoss()
#criterion = losses.TripletMarginLoss(margin=0.1)

loss_list_epoch_train = []
score_list_epoch_valid = []
score_best = 0
patience = 0
EPOCHS = 10
for epoch in range(EPOCHS):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=False)
    train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True)
    valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=False)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    loss_train, score_train = train_model(train_loader, model, optimizer, criterion) #, scheduler)
    #scheduler.step()
    out_valid = test_model(valid_loader, model)
    score_valid = make_score(valid, out_valid)

    loss_list_epoch_train.append(loss_train)
    score_list_epoch_valid.append(score_valid)

    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, train_acc: {score_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if score_valid > score_best:
        score_best = score_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience > 2:
            del train_loader, valid_loader, loss_train, out_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, out_valid
    torch.cuda.empty_cache()
    gc.collect()

acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.46875
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1, train_loss: 0.6931550158904153, train_acc: 0.49959935897435825, valid_score: 0.2002143767654097, time: 57.62519431114197
INFO:main:Epoch 1, train_loss: 0.6931550158904153, train_acc: 0.49959935897435825, valid_score: 0.2002143767654097, time: 57.62519431114197


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 2, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.89843964576721
INFO:main:Epoch 2, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.89843964576721


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 3, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.51547312736511
INFO:main:Epoch 3, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.51547312736511


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 4, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 56.95967102050781
INFO:main:Epoch 4, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 56.95967102050781


## 2回目学習

In [38]:
model = KDDModel().to(device)
model.load_state_dict(torch.load(MODEL_DIR+f'best_param.pt'))

for param in model.model_query().parameters():
    param.requires_grad = True
for param in model.model_image().parameters():
    param.requires_grad = False

TypeError: ignored

In [36]:
#model = KDDModel().to(device)

# if device == 'cuda':
#     model = torch.nn.DataParallel(model) # make parallel
#     cudnn.benchmark = True


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# optimizer = torch.optim.Adam([
#                         {'params': model.model_query.parameters()},
#                         {'params': model.model_image.parameters()}
#                     ])


criterion = nn.CrossEntropyLoss()
#criterion = nn.CosineSimilarity()
# criterion = nn.MSELoss()
#criterion = nn.CosineEmbeddingLoss()
#criterion = losses.TripletMarginLoss(margin=0.1)

loss_list_epoch_train = []
score_list_epoch_valid = []
score_best = 0
patience = 0
EPOCHS = 10
for epoch in range(EPOCHS):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=False)
    train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True)
    valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=False)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lambda epoch: 0.95 ** epoch)
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCHS)

    loss_train, score_train = train_model(train_loader, model, optimizer, criterion) #, scheduler)
    #scheduler.step()
    out_valid = test_model(valid_loader, model)
    score_valid = make_score(valid, out_valid)

    loss_list_epoch_train.append(loss_train)
    score_list_epoch_valid.append(score_valid)

    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, train_acc: {score_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if score_valid > score_best:
        score_best = score_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience > 2:
            del train_loader, valid_loader, loss_train, out_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, out_valid
    torch.cuda.empty_cache()
    gc.collect()

acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.57477569580078
INFO:main:Epoch 1, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.57477569580078


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 2, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 58.05427050590515
INFO:main:Epoch 2, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 58.05427050590515


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 3, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.407506465911865
INFO:main:Epoch 3, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.407506465911865


acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5
acc 0.5



Epoch 4, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.14650273323059
INFO:main:Epoch 4, train_loss: 0.6931474804878238, train_acc: 0.4999999999999992, valid_score: 0.2002143767654097, time: 57.14650273323059


## Inference

In [37]:
model = KDDModel().to(device)
model.load_state_dict(torch.load(MODEL_DIR+f'best_param.pt'))

<All keys matched successfully>

In [0]:
def make_submit_df(df_test, model):
    dataset_test = KDDDataset(df_test, train_mode=False)
    test_loader = DataLoader(dataset_test, batch_size=256, shuffle=False, num_workers=0, drop_last=False)
    out_list = test_model(test_loader, model)

    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    submit_df = pd.DataFrame(columns=['query-id','product1','product2','product3','product4','product5'])
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        submit_df.loc[i, 'query-id'] = q_id
        submit_df.iloc[i, 1:] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values
    submit_df = submit_df.astype(int)
    submit_df = submit_df.sort_values(by='query-id')
    submit_df = submit_df.reset_index(drop=True)
    
    return submit_df

In [0]:
submit_df = make_submit_df(testA, model)
submit_df.to_csv(SUBMIT_DIR+'submission.csv', index=False)